In [1]:
from keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [2]:
import tensorflow as tf

print("Before padding x_train.shape:", x_train.shape)  # Returns (60000, 28, 28)

# Pad image dimensions
x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]]) / 255

# Print padded image size
print("After padding x_train.shape:", x_train.shape)  # Returns (60000, 32, 32)

Before padding x_train.shape: (60000, 28, 28)
After padding x_train.shape: (60000, 32, 32)


In [3]:
# Add image channels
x_train = tf.stack([x_train] * 3, axis=-1)

# Print image dimensions after adding channels
print("After adding channels x_train.shape:", x_train.shape)

After adding channels x_train.shape: (60000, 32, 32, 3)


In [4]:
# Use the last 2000 images as validation set
x_val = x_train[-2000:]
y_val = y_train[-2000:]

# Use the rest of the images as training set
x_train = x_train[:-2000]
y_train = y_train[:-2000]

In [5]:
# Build base model
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Print base model structure information
base_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

In [6]:
from tensorflow.keras import layers, Model, losses

# Flatten base model output
x = layers.Flatten()(base_model.output)

# Define dense layer
x = layers.Dense(10, activation='softmax')(x)

# Build complete model
head_model = Model(inputs=base_model.input, outputs=x)

# Configure training parameters
head_model.compile(optimizer='adam',
                   loss=losses.sparse_categorical_crossentropy,
                   metrics=['accuracy'])

In [7]:
# Train model
batch_size = 64
epochs = 10
history = head_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

Epoch 1/10
907/907 [==============================] - 47s 35ms/step - loss: 0.4735 - accuracy: 0.8349 - val_loss: 0.0748 - val_accuracy: 0.9860
Epoch 2/10
907/907 [==============================] - 31s 34ms/step - loss: 0.0787 - accuracy: 0.9800 - val_loss: 0.0472 - val_accuracy: 0.9930
Epoch 3/10
907/907 [==============================] - 30s 34ms/step - loss: 0.0773 - accuracy: 0.9837 - val_loss: 0.0404 - val_accuracy: 0.9920
Epoch 4/10
907/907 [==============================] - 31s 34ms/step - loss: 0.0529 - accuracy: 0.9881 - val_loss: 0.0399 - val_accuracy: 0.9905
Epoch 5/10
907/907 [==============================] - 31s 34ms/step - loss: 0.0469 - accuracy: 0.9893 - val_loss: 0.0355 - val_accuracy: 0.9920
Epoch 6/10
907/907 [==============================] - 31s 34ms/step - loss: 0.0476 - accuracy: 0.9898 - val_loss: 0.0595 - val_accuracy: 0.9890
Epoch 7/10
907/907 [==============================] - 31s 34ms/step - loss: 0.0453 - accuracy: 0.9899 - val_loss: 0.0280 - val_accuracy:

In [10]:
# Select last 5000 test samples
x_test_selected = x_test[5000:]
y_test_selected = y_test[-5000:]

# Pad test images dimensions
x_test_selected = tf.pad(x_test_selected, [[0, 0], [2, 2], [2, 2]]) / 255

# Add test images channels
x_test_selected = tf.stack([x_test_selected] * 3, axis=-1)

# Test model performance
test_loss, test_accuracy = head_model.evaluate(x_test_selected, y_test_selected, verbose=2)
val_loss, val_accuracy = head_model.evaluate(x_val, y_val, verbose=1)

# Print test results
print("loss: {:.4f} - accuracy: {:.4f}".format(test_loss, test_accuracy))
print("Validation loss: {:.4f} - Validation accuracy: {:.4f}".format(val_loss, val_accuracy))

157/157 - 1s - loss: 0.0293 - accuracy: 0.9952 - 1s/epoch - 8ms/step
63/63 [==============================] - 1s 8ms/step - loss: 0.0320 - accuracy: 0.9945
loss: 0.0293 - accuracy: 0.9952
Validation loss: 0.0320 - Validation accuracy: 0.9945
